In [1]:
import random

import numpy as np
from sklearn.metrics import accuracy_score, top_k_accuracy_score

import torch
import pandas as pd
import tqdm

from transformers import DebertaTokenizerFast, DebertaForSequenceClassification, Trainer, TrainingArguments

from transformers import AutoTokenizer, AutoModelForSequenceClassification

from deberta_datasettmp import TextClassificationDataset
from torch.utils.data import DataLoader

In [2]:
pretrained_model_name = 'microsoft/deberta-v3-base'

model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name,
    num_labels=199
)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [3]:

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=False)
valid_ratio = 0.2
batch_size_per_device = 32
n_epochs = 10
warmup_ratio=0.2
max_length=256
num_classes=199

def get_datasets(data, valid_ratio=0.2):
    texts, labels = data['Text'].tolist(), data['MidIdx'].tolist()
    
    shuffeld = list(zip(texts, labels))
    random.shuffle(shuffeld)
  
    texts = [e[0] for e in shuffeld]
    labels = [e[1] for e in shuffeld]
    idx = int(len(texts) * (1 - valid_ratio))
    train_input, train_label = texts[:idx], labels[:idx]
    valid_input, valid_label = texts[idx:], labels[idx:]
    
    return  train_input, train_label, valid_input, valid_label


data = pd.read_csv('./traindata.csv')
train_input, train_label, valid_input, valid_label = get_datasets(data)

train_encodings = tokenizer(train_input, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_input, truncation=True, padding=True, max_length=max_length)
train_labels = torch.tensor(train_label)
valid_labels = torch.tensor(valid_label)

train_set = TextClassificationDataset(train_encodings,train_labels)
test_set = TextClassificationDataset(valid_encodings, valid_labels)

print(
    '|train|', len(train_input),
    '|valid|', len(valid_input),
)

total_batch_size = batch_size_per_device 
n_total_iterations = int(len(train_input) / total_batch_size * n_epochs) 
n_warmup_steps = int(n_total_iterations * warmup_ratio)

print(
    '#total_inters = ', n_total_iterations,
    '#warmup_iters = ', n_warmup_steps,
)



training_args = TrainingArguments(
    output_dir='./Deberta_V3_CheckPoint',
    num_train_epochs=n_epochs,
    per_device_train_batch_size=batch_size_per_device,
    per_device_eval_batch_size=batch_size_per_device,
    warmup_steps=n_warmup_steps,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_steps=n_total_iterations // 100,
    save_steps=n_total_iterations // n_epochs
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions

    return {
        'accuracy' : top_k_accuracy_score(labels, preds, k=2)
    }


# 이제 CustomTrainer를 사용하여 학습을 수행합니다.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


|train| 185266 |valid| 46317
#total_inters =  57895 #warmup_iters =  11579


In [5]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/root/storage/RNDClassification/deberta_datasettmp.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx]).long()


Epoch,Training Loss,Validation Loss,Accuracy
1,1.546600,1.410915,0.781182
2,1.212400,1.207976,0.822441
3,0.933200,0.993952,0.859058
4,0.772400,0.903953,0.877345
5,0.607100,0.917061,0.882419
6,0.491100,0.937121,0.885075
7,0.401600,0.953071,0.887817
8,0.303400,0.984900,0.888378
9,0.240500,1.048460,0.889026
10,0.173500,1.129718,0.888680


/root/storage/RNDClassification/deberta_datasettmp.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx]).long()
/root/storage/RNDClassification/deberta_datasettmp.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx]).long()
/root/storage/RNDClassification/deberta_datasettmp.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx]).long()
IOPub message rate exceeded.
The Jupyter server will temporarily st

TrainOutput(global_step=57900, training_loss=0.8438131112344113, metrics={'train_runtime': 37933.568, 'train_samples_per_second': 48.84, 'train_steps_per_second': 1.526, 'total_flos': 2.4416313635779584e+17, 'train_loss': 0.8438131112344113, 'epoch': 10.0})